In [ ]:
#import get to call a get request on the site
from requests import get
from bs4 import BeautifulSoup
from time import sleep
import re
from random import randint #avoid throttling by not sending too many requests one after the other
from warnings import warn
from time import time
from IPython.core.display import clear_output
import numpy as np


zipcodes=["02128", "02115", "02130", "02129", "02119", "02114", "02113", "02109", "02108", "02110", 
          "02210", "02116", "02111", "02127", "02118", "02199", "02215", "02120", "02125", "02122", 
          "02124", "02131", "02121", "02126", "02136", "02132", "02135", "02467", "02026", 
          "02134", "02446", "02445"]
post_timing = []
post_hoods = []
post_title_texts = []
bedroom_counts = []
sqfts = []
post_links = []
post_prices = []
zipcode =[]
for i in range(0, len(zipcodes)):
  #get the first page of the east bay housing prices
  response = get('https://boston.craigslist.org/d/apartments-housing-for-rent/search/apa?postal='+zipcodes[i])
  html_soup = BeautifulSoup(response.text, 'html.parser')

  #get the macro-container for the housing posts
  posts = html_soup.find_all('li', class_= 'result-row')
  #print(type(posts)) #to double check that I got a ResultSet
  print(len(posts))

  #find the total number of posts to find the limit of the pagination
  results_num = html_soup.find('div', class_= 'search-legend')
  results_total = int(results_num.find('span', class_='totalcount').text) #pulled the total count of posts as the upper bound of the pages array

  #each page has 119 posts so each new page is defined as follows: s=120, s=240, s=360, and so on. So we need to step in size 120 in the np.arange function
  pages = np.arange(0, results_total+1, 120)

  iterations = 0
  for page in pages:
    #get request
    response = get("https://boston.craigslist.org/d/apartments-housing-for-rent/search/apa?" 
                   + "s=" #the parameter for defining the page number 
                   + str(page)
                    + "&postal="+zipcodes[i]) #the page number in the pages array from earlier
                   #+ "&hasPic=1"
                   #+ "&availabilityMode=0")

    sleep(randint(1,5))
     
    #throw warning for status codes that are not 200
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))
        
    #define the html text
    page_html = BeautifulSoup(response.text, 'html.parser')
    
    #define the posts
    posts = html_soup.find_all('li', class_= 'result-row')
        
    #extract data item-wise
    for post in posts:

        if post.find('span', class_ = 'result-hood') is not None:

            #posting date
            #grab the datetime element 0 for date and 1 for time
            zipcode.append(zipcodes[i])
            post_datetime = post.find('time', class_= 'result-date')['datetime']
            post_timing.append(post_datetime)

            #neighborhoods
            post_hood = post.find('span', class_= 'result-hood').text
            post_hoods.append(post_hood)

            #title text
            post_title = post.find('a', class_='result-title hdrlnk')
            post_title_text = post_title.text
            post_title_texts.append(post_title_text)

            #post link
            post_link = post_title['href']
            post_links.append(post_link)
            
            #removes the \n whitespace from each side, removes the currency symbol, and turns it into an int
            if len(post.a.text.strip())!=0:
              post_price = post.a.text.strip().replace(",", "")
              post_price = int(post_price.replace("$", ""))
            else: 
              post_price=0
            post_prices.append(post_price)
            
            if post.find('span', class_ = 'housing') is not None:
                
                #if the first element is accidentally square footage
                if 'ft2' in post.find('span', class_ = 'housing').text.split()[0]:
                    
                    #make bedroom nan
                    bedroom_count = np.nan
                    bedroom_counts.append(bedroom_count)
                    
                    #make sqft the first element
                    sqft = int(post.find('span', class_ = 'housing').text.split()[0][:-3])
                    sqfts.append(sqft)
                    
                #if the length of the housing details element is more than 2
                elif len(post.find('span', class_ = 'housing').text.split()) > 2:
                    
                    #therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    bedroom_counts.append(bedroom_count)
                    
                    #and sqft will be number 3, so set these here and append
                    sqft = int(post.find('span', class_ = 'housing').text.split()[2][:-3])
                    sqfts.append(sqft)
                    
                #if there is num bedrooms but no sqft
                elif len(post.find('span', class_ = 'housing').text.split()) == 2:
                    
                    #therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    bedroom_counts.append(bedroom_count)
                    
                    #and sqft will be number 3, so set these here and append
                    sqft = np.nan
                    sqfts.append(sqft)                    
                
                else:
                    bedroom_count = np.nan
                    bedroom_counts.append(bedroom_count)
                
                    sqft = np.nan
                    sqfts.append(sqft)
                
            #if none of those conditions catch, make bedroom nan, this won't be needed    
            else:
                bedroom_count = np.nan
                bedroom_counts.append(bedroom_count)
                
                sqft = np.nan
                sqfts.append(sqft)
            #   bedroom_counts.append(bedroom_count)
                
            #   sqft = np.nan
            #   sqfts.append(sqft)
                
    iterations += 1
    print("Page " + str(iterations) + " scraped successfully!")

print("\n")

print("Scrape complete!")

42
Page 1 scraped successfully!
83
Page 1 scraped successfully!
120
Page 1 scraped successfully!
Page 2 scraped successfully!
28
Page 1 scraped successfully!
38
Page 1 scraped successfully!
46
Page 1 scraped successfully!
57
Page 1 scraped successfully!
15
Page 1 scraped successfully!
9
Page 1 scraped successfully!
9
Page 1 scraped successfully!
46
Page 1 scraped successfully!
47
Page 1 scraped successfully!
41
Page 1 scraped successfully!
67
Page 1 scraped successfully!
50
Page 1 scraped successfully!
41
Page 1 scraped successfully!
90
Page 1 scraped successfully!
36
Page 1 scraped successfully!
58
Page 1 scraped successfully!
45
Page 1 scraped successfully!
66
Page 1 scraped successfully!
30
Page 1 scraped successfully!
25
Page 1 scraped successfully!
5
Page 1 scraped successfully!
12
Page 1 scraped successfully!
11
Page 1 scraped successfully!
120
Page 1 scraped successfully!
Page 2 scraped successfully!
11
Page 1 scraped successfully!
5
Page 1 scraped successfully!
120
Page 1 scrap

In [ ]:
import pandas as pd

eb_apts = pd.DataFrame({'posted': post_timing,
                       'neighborhood': post_hoods,
                       'post title': post_title_texts,
                       'number bedrooms': bedroom_counts,
                        'sqft': sqfts,
                        'URL': post_links,
                       'price': post_prices,
                        'Zipcode': zipcode})
print(eb_apts.info())
eb_apts.head(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1681 entries, 0 to 1680
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   posted           1681 non-null   object 
 1   neighborhood     1681 non-null   object 
 2   post title       1681 non-null   object 
 3   number bedrooms  1491 non-null   object 
 4   sqft             764 non-null    float64
 5   URL              1681 non-null   object 
 6   price            1681 non-null   int64  
 7   Zipcode          1681 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 105.2+ KB
None


,posted,neighborhood,post title,number bedrooms,sqft,URL,price,Zipcode
0,2021-12-06 08:45,(East Boston boston/cambridge/brookline ),1 MONTH FREE RENT. Available NOW. Edison Lofts...,1,NaN,https://boston.craigslist.org/gbs/apa/d/boston...,2030,02128
1,2021-12-05 21:30,( boston/cambridge/brookline ),"Secure bike parking, Poolside restaurant servi...",1,687.0,https://boston.craigslist.org/gbs/apa/d/boston...,3600,02128
2,2021-12-05 21:28,( boston/cambridge/brookline ),"Access to boat landing, Music room, Energy Sta...",NaN,478.0,https://boston.craigslist.org/gbs/apa/d/boston...,2850,02128
3,2021-12-05 21:26,( boston/cambridge/brookline ),"Rooftop deck with seating and 270° views, Pet-...",NaN,396.0,https://boston.craigslist.org/gbs/apa/d/boston...,2525,02128
4,2021-12-05 21:24,( boston/cambridge/brookline ),"Low-flow fixtures, Nest Learning Thermostats™,...",NaN,476.0,https://boston.craigslist.org/gbs/apa/d/boston...,2650,02128
5,2021-12-05 20:30,( boston/cambridge/brookline ),"Access to a new waterfront park, Lounge with a...",1,567.0,https://boston.craigslist.org/gbs/apa/d/boston...,2700,02128
6,2021-12-04 12:36,(East Boston boston/cambridge/brookline ),Clean 1br ~ Parking ~ Public transit ~ Laundry,1,NaN,https://boston.craigslist.org/gbs/apa/d/east-b...,1600,02128
7,2021-12-02 13:35,(East Boston boston/cambridge/brookline ),FREE JANUARY RENT! Brand New Construction! No ...,3,NaN,https://boston.craigslist.org/gbs/apa/d/east-b...,3450,02128
8,2021-12-02 11:52,"(Orient Heights, Top of the Hill, Madonna Are...","$1750 1BR East Boston, Orient Heights, Madonna...",1,NaN,https://boston.craigslist.org/gbs/apa/d/east-b...,1750,02128
9,2021-12-01 18:22,( boston/cambridge/brookline ),"Rooftop deck with seating and 270° views, Sun ...",2,917.0,https://boston.craigslist.org/gbs/apa/d/boston...,4150,02128


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#drive.flush_and_unmount()
%cd "/content/drive/My Drive/Colab Notebooks"

eb_apts.to_csv('Boston_Craigslist_all_zips.csv')

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks


In [ ]:
import pandas as pd
eb_apts['Zipcode'] = pd.to_numeric(eb_apts['Zipcode'])
eb_apts['number bedrooms'] = pd.to_numeric(eb_apts['number bedrooms'])
eb_apts['Price per bedroom'] = eb_apts['price']/eb_apts['number bedrooms']

In [ ]:
univ_codes = [2115, 2215, 2116, 2108, 2125]
eb_apts['LOCATION_TYPE'] = eb_apts['Zipcode'].apply(lambda x: 'University' if x in univ_codes else 'Non-university')

In [ ]:
import numpy as np

Univ_values = eb_apts[eb_apts['LOCATION_TYPE']=='University']['price']
Non_univ_values = eb_apts[eb_apts['LOCATION_TYPE']=='Non-university']['price']

Univ_mean = np.mean(Univ_values)
Non_univ_mean = np.mean(Non_univ_values)
print("Univ mean value:",Univ_mean)
print("Non-Univ mean value:",Non_univ_mean)
Univ_std = np.std(Univ_values)
Non_univ_std = np.std(Non_univ_values)
print("Univ std value:",Univ_std)
print("Non-univ std value:",Non_univ_std)

Univ mean value: 2907.010752688172
Non-Univ mean value: 2782.698288159772
Univ std value: 1200.5916327800985
Non-univ std value: 1209.5391947964536


In [ ]:
from scipy.stats import ttest_ind
ttest,pval1 = ttest_ind(Univ_values,Non_univ_values)
print("t statistic",ttest)
print("p-value",float(pval1))

t statistic 1.5687739572003871
p-value 0.11688904727340975


In [ ]:
import numpy as np

Univ_values = eb_apts[eb_apts['LOCATION_TYPE']=='University']['Price per bedroom'].dropna()
Non_univ_values = eb_apts[eb_apts['LOCATION_TYPE']=='Non-university']['Price per bedroom'].dropna()

Univ_mean = np.mean(Univ_values)
Non_univ_mean = np.mean(Non_univ_values)
print("Univ mean value:",Univ_mean)
print("Non-Univ mean value:",Non_univ_mean)
Univ_std = np.std(Univ_values)
Non_univ_std = np.std(Non_univ_values)
print("Univ std value:",Univ_std)
print("Non-univ std value:",Non_univ_std)
from scipy.stats import ttest_ind
ttest,pval1 = ttest_ind(Univ_values,Non_univ_values)
print("t statistic",ttest)
print("p-value",float(pval1))

Univ mean value: 2101.2385290148445
Non-Univ mean value: 1606.3566356606943
Univ std value: 1190.2403952690127
Non-univ std value: 839.2270218522004
t statistic 7.829088021746101
p-value 9.264200284352637e-15
